In [1]:
!pip install rake-nltk

In [2]:
import pandas as pd
import numpy as np
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
df=pd.read_csv("IMDb movies.csv")


C:\Users\vkart\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.dropna()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
506,tt0017136,Metropolis,Metropolis,1927,1928-10-01,"Drama, Sci-Fi",153,Germany,German,Fritz Lang,...,"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rog...",In a futuristic city sharply divided between t...,8.3,156076,DEM 6000000,$ 1236166,$ 1349711,98.0,495.0,208.0
1048,tt0021749,Luci della città,City Lights,1931,1931-04-02,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,...,"Virginia Cherrill, Florence Lee, Harry Myers, ...","With the aid of a wealthy erratic tippler, a d...",8.5,162668,$ 1500000,$ 19181,$ 46008,99.0,295.0,122.0
2454,tt0027977,Tempi moderni,Modern Times,1936,1937-03-12,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,...,"Charles Chaplin, Paulette Goddard, Henry Bergm...",The Tramp struggles to live in modern industri...,8.5,211250,$ 1500000,$ 163577,$ 457688,96.0,307.0,115.0
2795,tt0029453,Il bandito della Casbah,Pépé le Moko,1937,1937-10-22,"Crime, Drama, Romance",94,France,"French, Arabic",Julien Duvivier,...,"Jean Gabin, Gabriel Gabrio, Saturnin Fabre, Fe...",A wanted gangster is both king and prisoner of...,7.7,6180,$ 60000,$ 155895,$ 155895,98.0,46.0,55.0
2827,tt0029583,Biancaneve e i sette nani,Snow White and the Seven Dwarfs,1937,1938-11-30,"Animation, Family, Fantasy",83,USA,English,"William Cottrell, David Hand",...,"Roy Atwell, Stuart Buchanan, Adriana Caselotti...",Exiled into the dangerous forest by her wicked...,7.6,177157,$ 1499000,$ 184925486,$ 184925486,95.0,260.0,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85266,tt9208444,Impractical Jokers: The Movie,Impractical Jokers: The Movie,2020,2020-02-28,Comedy,92,USA,English,Chris Henchy,...,"Paula Abdul, Joe Gatto, James Murray, Brian Qu...",The story of a humiliating high school mishap ...,5.9,4105,$ 3000000,$ 10691091,$ 10691091,39.0,152.0,17.0
85274,tt9214832,Emma.,Emma.,2020,2020-03-27,"Comedy, Drama",124,UK,English,Autumn de Wilde,...,"Anya Taylor-Joy, Angus Imrie, Letty Thomas, Ge...","In 1800s England, a well meaning but selfish y...",6.7,19858,$ 10000000,$ 10055355,$ 25659965,71.0,314.0,188.0
85397,tt9354944,Jexi,Jexi,2019,2019-10-11,"Comedy, Romance",84,"USA, Canada",English,"Jon Lucas, Scott Moore",...,"Adam Devine, Alexandra Shipp, Rose Byrne, Ron ...",A comedy about what can happen when you love y...,6.1,17038,$ 5000000,$ 6546159,$ 9341824,39.0,234.0,42.0
85460,tt9426210,Weathering with You,Tenki no ko,2019,2019-10-14,"Animation, Drama, Family",112,"Japan, China",Japanese,Makoto Shinkai,...,"Kotaro Daigo, Nana Mori, Shun Oguri, Sei Hirai...",A high-school boy who has run away to Tokyo be...,7.6,16277,$ 11100000,$ 7798743,$ 193176979,72.0,177.0,110.0


In [5]:
df=df.sample(20000)

In [6]:
df.describe()

,duration,avg_vote,votes,metascore,reviews_from_users,reviews_from_critics
count,20000.00000,20000.000000,2.000000e+04,3118.000000,18220.000000,17203.000000
mean,100.49145,5.883760,9.277494e+03,55.608082,45.205763,27.242400
std,22.86490,1.236441,5.347817e+04,17.728960,170.327259,58.229595
min,45.00000,1.000000,9.900000e+01,1.000000,1.000000,1.000000
25%,88.00000,5.200000,2.020000e+02,43.000000,4.000000,3.000000
50%,96.00000,6.100000,4.800000e+02,57.000000,9.000000,8.000000
75%,108.00000,6.800000,1.789000e+03,68.000000,27.000000,24.000000
max,729.00000,9.800000,2.002816e+06,100.000000,7639.000000,999.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 82022 to 16152
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_title_id          20000 non-null  object 
 1   title                  20000 non-null  object 
 2   original_title         20000 non-null  object 
 3   year                   20000 non-null  object 
 4   date_published         20000 non-null  object 
 5   genre                  20000 non-null  object 
 6   duration               20000 non-null  int64  
 7   country                19981 non-null  object 
 8   language               19798 non-null  object 
 9   director               19978 non-null  object 
 10  writer                 19626 non-null  object 
 11  production_company     18941 non-null  object 
 12  actors                 19979 non-null  object 
 13  description            19486 non-null  object 
 14  avg_vote               20000 non-null  float64
 15

In [8]:
df['genre'] = df['genre'].map(lambda x: str(x).lower().split(','))
df['actors'] = df['actors'].map(lambda x: str(x).split(',')[:2])
df['director'] = df['director'].map(lambda x: str(x).split(' '))

In [9]:
df.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
82022,tt7188742,Paisa Vasool,Paisa Vasool,2017,2017-09-01,"[action, comedy]",138,India,Telugu,"[Puri, Jagannadh]",...,"[Nandamuri Balakrishna, Shriya Saran]",A local officer recruits a deadly gangster to ...,4.4,502,NaN,NaN,$ 47245,NaN,1.0,7.0
52989,tt1043877,X-Cross,XX (ekusu kurosu): makyô densetsu,2007,2007-12-01,"[action, adventure, comedy]",90,Japan,Japanese,"[Kenta, Fukasaku]",...,"[Nao Matsushita, Ami Suzuki]","Recovering from a failed love affair, Shiyori ...",5.9,546,NaN,NaN,$ 416542,NaN,10.0,31.0
61388,tt1729674,18 anni scomparsa,Gone Missing,2013,2013-06-15,"[adventure, crime, drama]",90,USA,English,"[Tara, Miele]",...,"[Daphne Zuniga, Lauren Bowles]",When two best friends go missing during Spring...,5.4,788,NaN,NaN,NaN,NaN,10.0,1.0
32311,tt0134664,Go do gaai bei,Go do gaai bei,1997,1997-07-18,"[action, crime, thriller]",98,Hong Kong,"Mandarin, Cantonese","[Ringo, Lam]",...,"[Ching Wan Lau, Francis Ng]",Police Inspector Pao is trying to catch Mak Kw...,6.8,930,NaN,NaN,NaN,NaN,5.0,27.0
72549,tt3759912,Ladyworld,Ladyworld,2018,2019-10-18,[thriller],93,USA,English,"[Amanda, Kramer]",...,"[Ariela Barer, Annalise Basso]",Eight teenage girls become trapped in an endle...,3.5,337,NaN,NaN,$ 5413,54.0,15.0,21.0


In [10]:
df.tail()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
57088,tt1334328,Psyhi vathia,Psyhi vathia,2009,2009-10-22,"[drama, war]",142,"Greece, Cyprus",Greek,"[Pantelis, Voulgaris]",...,"[Vangelis Mourikis, Giorgos Symeonidis]",Near the end of the Greek Civil War two brothe...,7.1,2219,EUR 2,NaN,$ 2128154,NaN,8.0,4.0
72929,tt3859076,Truth: Il prezzo della verità,Truth,2015,2016-03-17,"[biography, drama, history]",125,"Australia, USA",English,"[James, Vanderbilt]",...,"[Cate Blanchett, Robert Redford]",Newsroom drama detailing the 2004 CBS 60 Minut...,6.8,18415,$ 9600000,$ 2541854,$ 5383097,66.0,115.0,193.0
18579,tt0077992,"The Night, the Prowler","The Night, the Prowler",1978,1979-06-15,"[drama, comedy]",90,Australia,English,"[Jim, Sharman]",...,"[Ruth Cracknell, John Frawley]","The Night The Prowler, is about the dark side ...",5.6,157,NaN,NaN,NaN,NaN,5.0,4.0
76223,tt4938790,Rüzgârin Hatiralari,Rüzgârin Hatiralari,2015,2015-12-11,[drama],124,"Turkey, Germany, France, Georgia",Turkish,"[Özcan, Alper]",...,"[Onur Saylak, Sofiya Khandamirova]",Addresses the issues of ethnicity and belongin...,6.4,756,NaN,NaN,NaN,NaN,2.0,10.0
16152,tt0070396,Al otro lado del espejo,Al otro lado del espejo,1973,1973,"[drama, horror]",82,"Spain, France, Portugal",Spanish,"[Jesús, Franco]",...,"[Emma Cohen, Robert Woods]",A nightclub singer is haunted by the ghost of ...,5.6,238,NaN,NaN,NaN,NaN,10.0,11.0


In [11]:
df.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
82022,tt7188742,Paisa Vasool,Paisa Vasool,2017,2017-09-01,"[action, comedy]",138,India,Telugu,"[Puri, Jagannadh]",...,"[Nandamuri Balakrishna, Shriya Saran]",A local officer recruits a deadly gangster to ...,4.4,502,NaN,NaN,$ 47245,NaN,1.0,7.0
52989,tt1043877,X-Cross,XX (ekusu kurosu): makyô densetsu,2007,2007-12-01,"[action, adventure, comedy]",90,Japan,Japanese,"[Kenta, Fukasaku]",...,"[Nao Matsushita, Ami Suzuki]","Recovering from a failed love affair, Shiyori ...",5.9,546,NaN,NaN,$ 416542,NaN,10.0,31.0
61388,tt1729674,18 anni scomparsa,Gone Missing,2013,2013-06-15,"[adventure, crime, drama]",90,USA,English,"[Tara, Miele]",...,"[Daphne Zuniga, Lauren Bowles]",When two best friends go missing during Spring...,5.4,788,NaN,NaN,NaN,NaN,10.0,1.0
32311,tt0134664,Go do gaai bei,Go do gaai bei,1997,1997-07-18,"[action, crime, thriller]",98,Hong Kong,"Mandarin, Cantonese","[Ringo, Lam]",...,"[Ching Wan Lau, Francis Ng]",Police Inspector Pao is trying to catch Mak Kw...,6.8,930,NaN,NaN,NaN,NaN,5.0,27.0
72549,tt3759912,Ladyworld,Ladyworld,2018,2019-10-18,[thriller],93,USA,English,"[Amanda, Kramer]",...,"[Ariela Barer, Annalise Basso]",Eight teenage girls become trapped in an endle...,3.5,337,NaN,NaN,$ 5413,54.0,15.0,21.0


In [12]:
df= df.drop(columns="date_published")

In [13]:
df.head()

,imdb_title_id,title,original_title,year,genre,duration,country,language,director,writer,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
82022,tt7188742,Paisa Vasool,Paisa Vasool,2017,"[action, comedy]",138,India,Telugu,"[Puri, Jagannadh]","Puri Jagannadh, A. Sreedhar",...,"[Nandamuri Balakrishna, Shriya Saran]",A local officer recruits a deadly gangster to ...,4.4,502,NaN,NaN,$ 47245,NaN,1.0,7.0
52989,tt1043877,X-Cross,XX (ekusu kurosu): makyô densetsu,2007,"[action, adventure, comedy]",90,Japan,Japanese,"[Kenta, Fukasaku]","Nobuyuki Jôkô, Tetsuya Oishi",...,"[Nao Matsushita, Ami Suzuki]","Recovering from a failed love affair, Shiyori ...",5.9,546,NaN,NaN,$ 416542,NaN,10.0,31.0
61388,tt1729674,18 anni scomparsa,Gone Missing,2013,"[adventure, crime, drama]",90,USA,English,"[Tara, Miele]",Bryan Dick,...,"[Daphne Zuniga, Lauren Bowles]",When two best friends go missing during Spring...,5.4,788,NaN,NaN,NaN,NaN,10.0,1.0
32311,tt0134664,Go do gaai bei,Go do gaai bei,1997,"[action, crime, thriller]",98,Hong Kong,"Mandarin, Cantonese","[Ringo, Lam]","Ringo Lam, Ringo Lam",...,"[Ching Wan Lau, Francis Ng]",Police Inspector Pao is trying to catch Mak Kw...,6.8,930,NaN,NaN,NaN,NaN,5.0,27.0
72549,tt3759912,Ladyworld,Ladyworld,2018,[thriller],93,USA,English,"[Amanda, Kramer]","Amanda Kramer, Benjamin Shearn",...,"[Ariela Barer, Annalise Basso]",Eight teenage girls become trapped in an endle...,3.5,337,NaN,NaN,$ 5413,54.0,15.0,21.0


In [14]:
df= df.drop(columns=["year", "writer" , "original_title" , "language"])
df.head()

,imdb_title_id,title,genre,duration,country,director,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
82022,tt7188742,Paisa Vasool,"[action, comedy]",138,India,"[Puri, Jagannadh]",Bhavya Creations,"[Nandamuri Balakrishna, Shriya Saran]",A local officer recruits a deadly gangster to ...,4.4,502,NaN,NaN,$ 47245,NaN,1.0,7.0
52989,tt1043877,X-Cross,"[action, adventure, comedy]",90,Japan,"[Kenta, Fukasaku]",Avex Entertainment,"[Nao Matsushita, Ami Suzuki]","Recovering from a failed love affair, Shiyori ...",5.9,546,NaN,NaN,$ 416542,NaN,10.0,31.0
61388,tt1729674,18 anni scomparsa,"[adventure, crime, drama]",90,USA,"[Tara, Miele]",Protocol Entertainment,"[Daphne Zuniga, Lauren Bowles]",When two best friends go missing during Spring...,5.4,788,NaN,NaN,NaN,NaN,10.0,1.0
32311,tt0134664,Go do gaai bei,"[action, crime, thriller]",98,Hong Kong,"[Ringo, Lam]",Brilliant Idea Group (BIG),"[Ching Wan Lau, Francis Ng]",Police Inspector Pao is trying to catch Mak Kw...,6.8,930,NaN,NaN,NaN,NaN,5.0,27.0
72549,tt3759912,Ladyworld,[thriller],93,USA,"[Amanda, Kramer]",Love & Death Productions (LDP),"[Ariela Barer, Annalise Basso]",Eight teenage girls become trapped in an endle...,3.5,337,NaN,NaN,$ 5413,54.0,15.0,21.0


In [15]:
df.drop(df.columns.difference(['title','imdb_title_id' , 'genre' , 'director' , 'actors' , 'description']), 1, inplace=True)

In [16]:
df.head()

,imdb_title_id,title,genre,director,actors,description
82022,tt7188742,Paisa Vasool,"[action, comedy]","[Puri, Jagannadh]","[Nandamuri Balakrishna, Shriya Saran]",A local officer recruits a deadly gangster to ...
52989,tt1043877,X-Cross,"[action, adventure, comedy]","[Kenta, Fukasaku]","[Nao Matsushita, Ami Suzuki]","Recovering from a failed love affair, Shiyori ..."
61388,tt1729674,18 anni scomparsa,"[adventure, crime, drama]","[Tara, Miele]","[Daphne Zuniga, Lauren Bowles]",When two best friends go missing during Spring...
32311,tt0134664,Go do gaai bei,"[action, crime, thriller]","[Ringo, Lam]","[Ching Wan Lau, Francis Ng]",Police Inspector Pao is trying to catch Mak Kw...
72549,tt3759912,Ladyworld,[thriller],"[Amanda, Kramer]","[Ariela Barer, Annalise Basso]",Eight teenage girls become trapped in an endle...


In [17]:
for index, row in df.iterrows():
    row['actors'] = [x.lower().replace(' ','') for x in row['actors']]
    row['director'] = ''.join(row['director']).lower()

In [18]:
df.head()

,imdb_title_id,title,genre,director,actors,description
82022,tt7188742,Paisa Vasool,"[action, comedy]",purijagannadh,"[nandamuribalakrishna, shriyasaran]",A local officer recruits a deadly gangster to ...
52989,tt1043877,X-Cross,"[action, adventure, comedy]",kentafukasaku,"[naomatsushita, amisuzuki]","Recovering from a failed love affair, Shiyori ..."
61388,tt1729674,18 anni scomparsa,"[adventure, crime, drama]",taramiele,"[daphnezuniga, laurenbowles]",When two best friends go missing during Spring...
32311,tt0134664,Go do gaai bei,"[action, crime, thriller]",ringolam,"[chingwanlau, francisng]",Police Inspector Pao is trying to catch Mak Kw...
72549,tt3759912,Ladyworld,[thriller],amandakramer,"[arielabarer, annalisebasso]",Eight teenage girls become trapped in an endle...


In [19]:
df['description']=df['description'].apply(str)

In [20]:
df['Key_words'] = ''
r = Rake() 
for index, row in df.iterrows():
    r.extract_keywords_from_text(row['description'])
    key_words_dict_scores = r.get_word_degrees()
    row['Key_words'] = list(key_words_dict_scores.keys())

In [21]:
df.head()

,imdb_title_id,title,genre,director,actors,description,Key_words
82022,tt7188742,Paisa Vasool,"[action, comedy]",purijagannadh,"[nandamuribalakrishna, shriyasaran]",A local officer recruits a deadly gangster to ...,"[crime, lord, local, officer, recruits, deadly..."
52989,tt1043877,X-Cross,"[action, adventure, comedy]",kentafukasaku,"[naomatsushita, amisuzuki]","Recovering from a failed love affair, Shiyori ...","[tranquil, village, best, friend, aiko, head, ..."
61388,tt1729674,18 anni scomparsa,"[adventure, crime, drama]",taramiele,"[daphnezuniga, laurenbowles]",When two best friends go missing during Spring...,"[everything, led, mothers, two, best, friends,..."
32311,tt0134664,Go do gaai bei,"[action, crime, thriller]",ringolam,"[chingwanlau, francisng]",Police Inspector Pao is trying to catch Mak Kw...,"[kwan, k, first, arrested, police, inspector, ..."
72549,tt3759912,Ladyworld,[thriller],amandakramer,"[arielabarer, annalisebasso]",Eight teenage girls become trapped in an endle...,"[psyches, dissolve, water, food, eight, teenag..."


In [22]:
df.drop('imdb_title_id', axis=1)

,title,genre,director,actors,description,Key_words
82022,Paisa Vasool,"[action, comedy]",purijagannadh,"[nandamuribalakrishna, shriyasaran]",A local officer recruits a deadly gangster to ...,"[crime, lord, local, officer, recruits, deadly..."
52989,X-Cross,"[action, adventure, comedy]",kentafukasaku,"[naomatsushita, amisuzuki]","Recovering from a failed love affair, Shiyori ...","[tranquil, village, best, friend, aiko, head, ..."
61388,18 anni scomparsa,"[adventure, crime, drama]",taramiele,"[daphnezuniga, laurenbowles]",When two best friends go missing during Spring...,"[everything, led, mothers, two, best, friends,..."
32311,Go do gaai bei,"[action, crime, thriller]",ringolam,"[chingwanlau, francisng]",Police Inspector Pao is trying to catch Mak Kw...,"[kwan, k, first, arrested, police, inspector, ..."
72549,Ladyworld,[thriller],amandakramer,"[arielabarer, annalisebasso]",Eight teenage girls become trapped in an endle...,"[psyches, dissolve, water, food, eight, teenag..."
...,...,...,...,...,...,...
57088,Psyhi vathia,"[drama, war]",pantelisvoulgaris,"[vangelismourikis, giorgossymeonidis]",Near the end of the Greek Civil War two brothe...,"[opposite, sides, end, near, greek, civil, war..."
72929,Truth: Il prezzo della verità,"[biography, drama, history]",jamesvanderbilt,"[cateblanchett, robertredford]",Newsroom drama detailing the 2004 CBS 60 Minut...,"[president, george, w, criticism, careers, pro..."
18579,"The Night, the Prowler","[drama, comedy]",jimsharman,"[ruthcracknell, johnfrawley]","The Night The Prowler, is about the dark side ...","[class, urban, culture, suburban, family, life..."
76223,Rüzgârin Hatiralari,[drama],özcanalper,"[onursaylak, sofiyakhandamirova]",Addresses the issues of ethnicity and belongin...,"[conveys, across, borders, exist, feels, count..."


In [23]:
df['Bag_of_words'] = ''
columns = ['genre', 'director', 'actors', 'Key_words']
for index, row in df.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col]) + ' '
    row['Bag_of_words'] = words
    
df = df[['title','Bag_of_words']]

In [24]:
df.head()

,title,Bag_of_words
82022,Paisa Vasool,action comedy p u r i j a g a n n a d h nanda...
52989,X-Cross,action adventure comedy k e n t a f u k a s ...
61388,18 anni scomparsa,adventure crime drama t a r a m i e l e daph...
32311,Go do gaai bei,action crime thriller r i n g o l a m chingw...
72549,Ladyworld,thriller a m a n d a k r a m e r arielabarer a...


In [25]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['Bag_of_words'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

[[1.         0.12038585 0.06154575 ... 0.06299408 0.         0.        ]
 [0.12038585 1.         0.08891084 ... 0.04550158 0.04783649 0.        ]
 [0.06154575 0.08891084 1.         ... 0.04652421 0.0489116  0.0489116 ]
 ...
 [0.06299408 0.04550158 0.04652421 ... 1.         0.05006262 0.05006262]
 [0.         0.04783649 0.0489116  ... 0.05006262 1.         0.05263158]
 [0.         0.         0.0489116  ... 0.05006262 0.05263158 1.        ]]


In [26]:
indices = pd.Series(df['title'])

In [91]:
def recommend(title):
    recommended_movies = []
    idx = indices[indices == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_movies.append(list(df['title'])[i])
        
    return recommended_movies

In [123]:
recommend("Cleopatra")

['Solo andata',
 'Les voleurs de la nuit',
 'Radio Cab Murder',
 'La banda Bonnot',
 'Ladies of the Big House',
 'Jag är min egen',
 "Delitto sull'autostrada",
 'Hævnens Nat',
 'Borsalino',
 'Smart Alecks']

In [85]:
recommend("Eureka")

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
from tkinter import *
import tkinter as tk
from PIL import ImageTk,Image
win = tk.Tk()

canvas=Canvas(win,width=1650, height=850)
win.title("Movie-App")

entry1= tk.Entry(win)
canvas.create_window(200,140, window = entry1)
entry1.grid(row=0, column=1) 

T= tk.Text(win, height= 50, width = 60)



def myClick():
    title= entry1.get()
    
    recommended_movies = recommend(title)
    var = print(recommended_movies)
    #this creates a new label to the GUI
    T.insert(tk.END,var)
    

myButton = Button(win, text="Predict", width=18, command=myClick).place(x=800, y=150)


win.mainloop() 

['Solo andata', 'Les voleurs de la nuit', 'Radio Cab Murder', 'La banda Bonnot', 'Ladies of the Big House', 'Jag är min egen', "Delitto sull'autostrada", 'Hævnens Nat', 'Borsalino', 'Smart Alecks']


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\vkart\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-158-45bd7e8ecdc6>", line 23, in myClick
    T.insert(tk.END,var)
  File "C:\Users\vkart\anaconda3\lib\tkinter\__init__.py", line 3738, in insert
    self.tk.call((self._w, 'insert', index, chars) + args)
_tkinter.TclError: wrong # args: should be ".!text insert index chars ?tagList chars tagList ...?"


In [ ]:
#Convert list to string
#then print the output in text var :))

In [124]:
recommend("Al otro lado del espejo")

['Ranczo Wilkowyje',
 'Kútfejek',
 'The Microchip',
 'Kamen raidâ x Kamen raidâ Fôze & Ôzu Movie taisen Mega Max',
 'Weekend',
 'Che carambole... ragazzi!!!...',
 'Kolay para',
 'Donga Police',
 'Adavi Donga',
 'Peddarikam']